#**The Entreprenuership Network**

#**Core Task 4: Create a Image to emoji project.**

**Author Name : Teena Joseph(MSC DA), Rajagiri College of Social Sciences**

**Import the required libraries**

In [ ]:
import NumPy as np
import cv2
from Keras.emotion_models import Sequential
from kerasKeras.layers import Dense
from Keras.layers import Dropout
from Keras.layers import Flatten
from Keras. layers import Conv2D
from Keras.optimizers import Adam
from Keras. layers import MaxPooling2D
from Keras.preprocessing.image import ImageDataGenerator

**Initialize the training and validation generators:**

In [ ]:
train_dir = 'data/train'
val_dir = 'data/test'
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
#training generator for CNN
train_generator = train_datagen.flow_from_directory(
       train_dir,
       target_size=(48,48),
       batch_size=64,
       color_mode="gray_framescale",
       class_mode='categorical')
#validation generator for CNN
validation_generator = val_datagen.flow_from_directory(
       val_dir,
       target_size=(48,48),
       batch_size=64,
       color_mode="gray_framescale",
       class_mode='categorical')

Output

Found 28709 images belonging to 7 classes.

Found 7178 images belonging to 7 classes.

In [ ]:
#To display the train data

for i in os.listdir("train/"):
    print(str(len(os.listdir("train/"+i))) +" "+ i +" images")

Output
3995 angry images

436 disgust images

4097 fear images

7215 happy images

4965 neutral images

4830 sad images

3171 surprise images

In [ ]:
#To display the test data

for i in os.listdir("test/"):
    print(str(len(os.listdir("test/"+i))) +" "+ i +" images")

Output
958 angry images

111 disgust images

1024 fear images

1774 happy images

1233 neutral images

1247 sad images

831 surprise images

In [ ]:
#Build the convolution network architecture:

emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))#output=(48-3+0)/1+1=46
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))#output=(46-3+0)/1+1=44
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))#output=devided input by 2 it means 22,22,64
emotion_model.add(Dropout(0.25))#reduce 25% module at a time of output
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu',input_shape=(48,48,1)))#(22-3+0)/1+1=20
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))#10
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))#(10-3+0)/1+1=8
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))#output=4
emotion_model.add(Dropout(0.25))#nothing change
emotion_model.add(Flatten())#here we get multidimension output and pass as linear to the dense so that 4*4*128=2048
emotion_model.add(Dense(1024, activation='relu'))#hddien of 1024 neurons of input 
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))#hddien of 7 neurons of input
plot_model(emotion_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)#save model leyer as model_plot.png
emotion_model.summary()

In [ ]:
# Compile and train the model
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
emotion_model_info = emotion_model.fit_generator( #to fetch the model info from validation generator
       train_generator,
       steps_per_epoch=28709 // 64,
       epochs=50,
       validation_data=validation_generator,
       validation_steps=7178 // 64)

In [ ]:
#Save the model weights:
emotion_model.save_weights('model.h5')#to save the model

In [ ]:
#To detect bounding boxes of face in the webcam and to predict the emotions we use OpenCV Haarcascade xml:
cv2.ocl.setUseOpenCL(False)
#emotion dictionary creation
em_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
cap = cv2.VideoCapture(0)
while True:
   ret, fram = cap.read()
if not ret:
       break
#bounding box initialization  
 bounding_box = cv2.CascadeClassifier('cv2/data/haarcascade_frontalface_default.xml')
   gray_frame = cv2.cvtColor(fram, cv2.COLOR_BGR2gray_frame)
#to detect the multiple faces and frame them separately   
n_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)
for (x, y, w, h) in n_faces:
       cv2.rectangle(fram, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
       roi_frame = gray_frame[y:y + h, x:x + w]
       crop_img = np.expand_dims(np.expand_dims(cv2.resize(roi_frame, (48, 48)), -1), 0)
       emotion_prediction = emotion_model.predict(crop_img)
       maxindex = int(np.argmax(emotion_prediction))
       cv2.putText(frame, em_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
   cv2.imshow('Video', cv2.resize(frame,(1200,860),interpolation = cv2.INTER_CUBIC))
if cv2.waitKey(1) & 0xFF == ord('q'):
cap.release()
cv2.destroyAllWindows()
   break